In [1]:
import numpy as np
import pyvista as pv
import potpourri3d as pp3d

In [2]:
mesh = pv.read("/Users/patrykrygiel/Downloads/hdd-cell-t008.vtp")
field = mesh["wss"]

dec = pp3d.DifferentialExteriorCalculus(mesh.points, mesh.regular_faces)
one_form = dec.to_Edge1Form(field)
exact, coexact, harmonic = dec.hodge_decomposition(one_form)

print(f"One form: {one_form.sum()}")
for name, one_form in {"exact": exact, "coexact": coexact, "harmonic": harmonic}.items():
    print(f"Component: {name}")
    print(f"- divergence: {dec.divergence(one_form).sum()}")
    print(f"- curl: {dec.curl(one_form).sum()}")

mesh["exact"] = dec.to_Face2Form(exact)
mesh["coexact"] = dec.to_Face2Form(coexact)
mesh["harmonic"] = dec.to_Face2Form(harmonic)
mesh["wss-rec"] = mesh["exact"] + mesh["coexact"] + mesh["harmonic"]
mesh["wss-mae"] = np.abs(mesh["wss"] - mesh["wss-rec"])

mesh.save("hhd-py.vtp")

print(f"\nDecomposition MAE: {mesh['wss-mae'].mean()}")

for name in ["wss", "exact", "coexact", "harmonic"]:
    one_form = dec.to_Edge1Form(mesh[name])
    print(f"Component: {name}")
    print(f"- divergence: {dec.divergence(one_form).sum()}")
    print(f"- curl: {dec.curl(one_form).sum()}")

One form: -216.97035873903005
Component: exact
- divergence: 328556.7366106078
- curl: -3.650558007442245e-11
Component: coexact
- divergence: -5.367704209685636e-10
- curl: -429779.6441535887
Component: harmonic
- divergence: 4.169577653841766
- curl: -1.0928795804731098e-07

Decomposition MAE: 0.26626366691091274
Component: wss
- divergence: 328560.90618826123
- curl: -429779.6441536976
Component: exact
- divergence: 328556.73661060777
- curl: -1.4192444700842248e-10
Component: coexact
- divergence: 93363.80176925681
- curl: -346063.2204029245
Component: harmonic
- divergence: 4.169578726216284
- curl: -1.4345396907675802e-07


In [ ]:
pl = pv.Plotter(shape=(2, 3), window_size=(1200, 800))

# Reconstruction
pl.subplot(0, 0)
pl.add_text("Original WSS")
pl.add_mesh(mesh.copy(), scalars="wss", clim=[0, 30])

pl.subplot(0, 1)
pl.add_text("Reconstructed WSS")
pl.add_mesh(mesh.copy(), scalars="wss-rec", clim=[0, 30])

pl.subplot(0, 2)
pl.add_text("Reconstruction MAE")
pl.add_mesh(mesh.copy(), scalars="wss-mae", clim=[0, 5])

# Components
for i, name in enumerate(["exact", "coexact", "harmonic"]):
    pl.subplot(1, i)
    pl.add_text(name)
    pl.add_mesh(mesh.copy(), color="white", opacity=0.3)
    pl.add_arrows(
        mesh.cell_centers().points[::2], 
        (mesh[name] / np.linalg.norm(mesh[name], axis=1, keepdims=True))[::2], 
        mag=0.01,
        show_scalar_bar=False
    )

pl.link_views()
pl.show()

Widget(value='<iframe src="http://localhost:59066/index.html?ui=P_0x3395b2e10_2&reconnect=auto" class="pyvista…